# Imports

In [ ]:
import numpy as np

import cyclops.query.process as qp
from cyclops.processors.column_names import ENCOUNTER_ID, LENGTH_OF_STAY_IN_ER, DIAGNOSIS_CODE
from cyclops.query import gemini
from cyclops.query.gemini import get_interface

# Query

In [ ]:
def get_cohort():
    """Get delirium cohort."""
    table = gemini.patient_encounters().query
    table = qp.DropNulls("del_present")(table)
    table = qp.ConditionEquals("del_present", 3, not_=True)(table)

    return get_interface(table).run()


def get_room_transfers_for_cohort(cohort):
    """Get room transfer info for cohort."""
    table = gemini.room_transfers().query
    table = qp.GroupByAggregate(
        ENCOUNTER_ID, {ENCOUNTER_ID: ("count", "room_transfers")}
    )(table)
    room_transfers = get_interface(table).run()
    cohort = cohort.merge(room_transfers, on=ENCOUNTER_ID, how="left")

    return cohort


def get_scu_transfers_for_cohort(cohort):
    """Get SCU transfer info for cohort."""
    table = gemini.get_table("ip_scu")
    table = qp.GroupByAggregate(
        ENCOUNTER_ID, {ENCOUNTER_ID: ("count", "icu_transfers")}
    )(table)
    ip_scu = get_interface(table).run()
    cohort = cohort.merge(ip_scu, on=ENCOUNTER_ID, how="left")
    cohort["icu_transfers"] = cohort["icu_transfers"].fillna(0)

    return cohort


def get_er_for_cohort(cohort):
    """Get ER info for cohort."""
    table = gemini.er_admin().query
    table = qp.FilterColumns(
        [
            ENCOUNTER_ID,
            "admit_via_ambulance",
            "triage_level",
            LENGTH_OF_STAY_IN_ER,
        ]
    )(table)
    er_info = get_interface(table).run()
    cohort = cohort.merge(er_info, on=ENCOUNTER_ID, how="left")

    return cohort


def get_bt_for_cohort(cohort):
    """Get blood transfusions info for cohort."""
    table = gemini.blood_transfusions().query
    total = qp.GroupByAggregate(
        ENCOUNTER_ID, {ENCOUNTER_ID: ("count", "total_transfusions")}
    )(table)
    unique = qp.GroupByAggregate(
        [ENCOUNTER_ID, "blood_product_raw"], {ENCOUNTER_ID: ("count", "unique_transfusions")}
    )(table)
    unique = qp.GroupByAggregate(
        ENCOUNTER_ID, {ENCOUNTER_ID: ("count", "unique_transfusions")}
    )(unique)
    total = get_interface(total).run()
    unique = get_interface(unique).run()
    cohort = cohort.merge(total, on=ENCOUNTER_ID, how="left")
    cohort = cohort.merge(unique, on=ENCOUNTER_ID, how="left")
    cohort["unique_transfusions"] = cohort["unique_transfusions"].fillna(0)
    cohort["total_transfusions"] = cohort["total_transfusions"].fillna(0)

    return cohort


def get_icd_delirium_for_cohort(cohort):
    """Get ICD delirium flag based on diagnosis info for cohort."""
    icd_delirium = gemini.diagnoses().query
    # icd_delirium = qp.ConditionStartsWith(DIAGNOSIS_CODE, "F05")(icd_delirium)
    # icd_delirium = qp.GroupByAggregate(
    #     [ENCOUNTER_ID, DIAGNOSIS_CODE],  {ENCOUNTER_ID: ("count", "icd_delirium")}
    # )(table)
    # icd_delirium = qp.GroupByAggregate(
    #     [ENCOUNTER_ID, DIAGNOSIS_CODE], {ENCOUNTER_ID: ("count", "icd_delirium")}
    # )(icd_delirium)
    # icd_delirium = qp.FilterColumns(
    #     [
    #         ENCOUNTER_ID,
    #         DIAGNOSIS_CODE,
    #         "icd_delirium",
    #     ]
    # )(icd_delirium)
    icd_delirium = get_interface(icd_delirium).run()
    # cohort = cohort.merge(icd_delirium, on=ENCOUNTER_ID, how="left")
    # cohort["icd_delirium"] = cohort["icd_delirium"].fillna(0)

    return icd_delirium


cohort = get_cohort()
cohort = get_room_transfers_for_cohort(cohort)
cohort = get_scu_transfers_for_cohort(cohort)
cohort = get_er_for_cohort(cohort)
cohort = get_bt_for_cohort(cohort)
cohort = get_icd_delirium_for_cohort(cohort)

In [ ]:
cohort

In [ ]:
import psycopg2
import pandas as pd


conn = psycopg2.connect(
    host="db.gemini-hpc.ca",
    database="delirium_v3_0_0",
    user="krishnanam",
    password="Totem110#",
)

delirium_icd10_query = """
    select
        a.genc_id, a.diagnosis_code, bool_or(a.icd_delirium) as icd_delirium
    from (
        select distinct
            d.genc_id, d.diagnosis_code as diagnosis_code,
            case when d.diagnosis_code like 'F05%' then TRUE else FALSE end as icd_delirium
        from diagnosis d
            inner join ip_administrative ia
                on d.genc_id = ia.genc_id
        where ia.del_present is not null and ia.del_present <> 3
    ) a
    group by a.genc_id, a.diagnosis_code;
"""
ref = pd.read_sql_query(delirium_icd10_query, conn)

In [ ]:
ref_encs = set(list(ref[ref["icd_delirium"] == True]["genc_id"]))
tar_encs = set(list(cohort[cohort["icd_delirium"] == 1.0]["encounter_id"]))
inter = ref_encs.intersection(tar_encs)
len(inter)
ref_encs - tar_encs

In [ ]:
cohort[cohort["encounter_id"] == 15301942]

In [ ]:
ref[ref["genc_id"] == 15301942]